In [1]:
import pandas as pd
import numpy as np

In [211]:
pop_df = pd.read_csv("./West Coast 2023 Datasets/Traffic, Investigations _ Other/philadelphia_population_metrics.csv")

In [212]:
pop_df = pop_df.dropna(axis=0, how="any")
pop_df = pop_df.rename(columns={'GEOGRAPHY_NAME': "fips", 'COUNT_ALL_RACES_ETHNICITIES': 'count', 
                        "COUNT_WHITE_NH": "White - Non-Latino Count", "COUNT_BLACK_NH": "Black - Non-Latino Count",
                       "COUNT_ASIAN_NH": 'Asian Count', "COUNT_HISPANIC": 'Latino Count',
                               "PERCENT_WHITE_NH": "White - Non-Latino", "PERCENT_BLACK_NH": "Black - Non-Latino",
                               "PERCENT_ASIAN_NH": "Asian", "PERCENT_HISPANIC": "Latino"})

#pop_df.pivot(index="fips",columns="Race Percentage",values=["White - Non-Latino","Black - Non-Latino", "Asian", "Latino"])
pop_df = pop_df.drop(["OBJECTID", "GEOGRAPHY", "Shape__Area", "Shape__Length"
                     # 'Asian Count', 'Latino Count', "White - Non-Latino Count", "Black - Non-Latino Count"
                     ], axis=1)
pop_df2 = pd.melt(pop_df, id_vars=["fips"], value_vars=["Asian", "Latino", "White - Non-Latino", "Black - Non-Latino"])
print(pop_df)
pop_df2

            fips  count  White - Non-Latino Count  Black - Non-Latino Count  \
0    42101000100   3478                      2890                       207   
1    42101000200   2937                       665                       284   
2    42101000300   3169                      2290                       324   
3    42101000401   2125                      1049                       376   
4    42101000402   3142                      2455                       173   
..           ...    ...                       ...                       ...   
376  42101980300      2                         2                         0   
380  42101980700      5                         0                         3   
381  42101980800      6                         4                         1   
382  42101980900     30                         9                        18   
383  42101989100   7765                       975                      5469   

     Asian Count  Latino Count  White - Non-Latino 

,fips,variable,value
0,42101000100,Asian,4.974123
1,42101000200,Asian,63.159687
2,42101000300,Asian,10.350268
3,42101000401,Asian,24.423529
4,42101000402,Asian,11.330363
...,...,...,...
1519,42101980300,Black - Non-Latino,0.000000
1520,42101980700,Black - Non-Latino,60.000000
1521,42101980800,Black - Non-Latino,16.666667
1522,42101980900,Black - Non-Latino,60.000000


In [186]:
pop_df

,fips,count,White - Non-Latino Count,Black - Non-Latino Count,Asian Count,Latino Count,White - Non-Latino,Black - Non-Latino,Asian,Latino
0,42101000100,3478,2890,207,173,126,83.093732,5.951696,4.974123,3.622772
1,42101000200,2937,665,284,1855,79,22.642152,9.669731,63.159687,2.689820
2,42101000300,3169,2290,324,328,135,72.262543,10.224045,10.350268,4.260019
3,42101000401,2125,1049,376,519,107,49.364706,17.694118,24.423529,5.035294
4,42101000402,3142,2455,173,356,109,78.134946,5.506047,11.330363,3.469128
...,...,...,...,...,...,...,...,...,...,...
376,42101980300,2,2,0,0,0,100.000000,0.000000,0.000000,0.000000
380,42101980700,5,0,3,1,1,0.000000,60.000000,20.000000,20.000000
381,42101980800,6,4,1,0,0,66.666667,16.666667,0.000000,0.000000
382,42101980900,30,9,18,0,2,30.000000,60.000000,0.000000,6.666667


In [213]:
inv_df = pd.read_csv("./West Coast 2023 Datasets/Traffic, Investigations _ Other/investigations.csv")
len(inv_df)

2785087

In [214]:
inv_df["fips"] = inv_df["fips"].astype('Int64')
inv_df = inv_df.dropna(axis=0, how="any")
inv_df["fips"] = inv_df.apply(lambda row: int(row["fips"] / 10) if len(str(row["fips"])) > 11 else row["fips"], axis = 1)

In [215]:
inv_df.head(1)

,id,datetimeoccur,weekday,location,districtoccur,psa,stoptype,inside_or_outside,gender,race,...,individual_frisked,individual_searched,individual_arrested,individual_contraband,vehicle_frisked,vehicle_searched,vehicle_contraband,lat,lng,fips
0,1767776,2014-05-14 00:00:00+00,TUESDAY,300 BLOCK S 13TH ST,6,3,pedestrian,Outside,Male,Black - Non-Latino,...,0,0,0,0,0,0,0,39.946671,-75.162656,42101001101


In [216]:
inv_df["year"] = pd.DatetimeIndex(inv_df['datetimeoccur']).year

In [217]:
inv_groups = inv_df.groupby(["fips", "year", "race"]).size().reset_index(name="counts")
#inv_df.groupby(["fips", "year", "race"])
# pd.pivot_table(inv_df, index = ["fips", "year"], columns = "race", aggfun=np.count)

In [218]:
inv_year_count = inv_df.groupby(["fips", "year"]).size().reset_index(name="counts")


In [219]:
inv_groups["percentage"] = inv_groups.apply(lambda row: float(row["counts"] / 
                                                              inv_year_count.loc[(inv_year_count['year'] == row["year"])& (inv_year_count['fips'] == row["fips"])]["counts"] * 100)
                                                              , axis=1).T
                               #             (row["counts"] / 
                                          #  inv_year_count.loc[(inv_year_count['year'] == row["year"]) 
                                          #                     & (inv_year_count['fips'] == row["fips"])]["counts"]), axis=1).T

In [220]:
inv_year_count = inv_year_count[inv_year_count["counts"] > 10]


In [221]:
inv_all_percentage = pd.merge(inv_year_count, inv_groups, on=["fips", "year"], how="left")

In [222]:
inv_all_percentage["race"].unique()

array(['Asian', 'Black - Non-Latino', 'White - Latino',
       'White - Non-Latino', 'Unknown', 'American Indian',
       'Black - Latino'], dtype=object)

In [223]:
inv_vs_pop = pd.merge(inv_all_percentage, pop_df2, on="fips", how="left")

In [224]:
inv_vs_pop = inv_vs_pop.dropna(axis=0, how="any")

In [225]:
inv_vs_pop = inv_vs_pop[inv_vs_pop["race"] == inv_vs_pop["variable"]]

In [232]:
inv_vs_pop["representation"] = inv_vs_pop.apply(lambda row: row["percentage"] / row["value"] if row["value"] > 0 else 1, axis = 1)

In [239]:
inv_vs_pop.sort_values(by="representation", ascending=False)

,fips,year,counts_x,race,counts_y,percentage,variable,value,representation
29718,42101018002,2022,56,Black - Non-Latino,43,76.785714,Black - Non-Latino,0.605394,126.836038
29990,42101018400,2021,12,Black - Non-Latino,5,41.666667,Black - Non-Latino,0.353982,117.708334
29654,42101018002,2019,460,Black - Non-Latino,327,71.086957,Black - Non-Latino,0.605394,117.422726
29674,42101018002,2020,73,Black - Non-Latino,48,65.753425,Black - Non-Latino,0.605394,108.612702
29578,42101018002,2016,315,Black - Non-Latino,186,59.047619,Black - Non-Latino,0.605394,97.535930
...,...,...,...,...,...,...,...,...,...
9459,42101008801,2018,96,Asian,1,1.041667,Asian,30.161943,0.034536
30759,42101019100,2022,277,Asian,1,0.361011,Asian,10.671538,0.033829
9107,42101008701,2022,118,Asian,1,0.847458,Asian,25.488305,0.033249
5739,42101007700,2014,406,Asian,3,0.738916,Asian,24.495798,0.030165


In [241]:
inv_vs_pop.to_csv("./cleaned/inv_vs_pop.csv")

FileNotFoundError: [Errno 2] No such file or directory: './cleaned/inv_vs_pop.csv'